# MLOps or how not to die to put models in production

## MLFlow Tracking
This notebook shows some examples of using mlflow library to track useful information about machine learning experiments.

It is necessary to install mlflow (``pip install mlflow``) in the working environment or directly over a simple Python installation.

### Basic example
This example show a basic use of mlflow to start an experiment, track parameters and track metrics.

In [ ]:
import mlflow

In [ ]:
tracking_uri = "http://localhost:5000"
mlflow.set_tracking_uri(tracking_uri)

In general, it is not necessary to explicitly call ```mlflow.create_experiment('name')``` before setting it. We can just set the experiment and mlflow will create it if not existed before.

In [ ]:
mlflow.set_experiment('test_1')

In [ ]:
run =  mlflow.start_run()

Another alternative is:
```python
with mlflow.start_run() as run:
    ...
```
This way prevents calling ``mlflow.end_run()`` to finish the run.

In [ ]:
mlflow.log_param('param1', 1)
mlflow.log_metric('metric1', 2)

In [ ]:
mlflow.log_param('param1', 1)
mlflow.log_metric('metric1', 2)

In [ ]:
mlflow.end_run()

### Breast cancer: Scikit-learn
Now, some models will be trained on breast cancer dataset.

So, again, a new experiment is necessary for that.

In [ ]:
mlflow.set_experiment('breast_cancer')

In [ ]:
import numpy as np
import pandas
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [ ]:
cancer = load_breast_cancer()
cancer.keys()

In [ ]:
type(cancer)

In [ ]:
X = np.array(cancer.data)
y = np.array(cancer.target)
print(f'X: {X.shape}, y: {y.shape}')

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=426, test_size=143, random_state=0)

After splitting the dataset, it is necessary to apply a feature scaling to improve the model's results.

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

Next function computes three of the most used measures in machine learning.

In [ ]:
def validate_model(model, x_test, y_test):    
    y_pred = model.predict(x_test)
    y_pred = (y_pred > 0.5)
    from sklearn.metrics import confusion_matrix
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    
    return precision, recall, accuracy

First model relies on a Logistic Regression, which is the most used model as baseline for binary classification.

In [ ]:
# Model 1: Logistic Regression
def breast_cancer_lr(solver="lbfgs", C=1.0):
    from sklearn.linear_model import LogisticRegression
    import mlflow.sklearn
    with mlflow.start_run() as run:
        lr = LogisticRegression(solver = solver, C = C)
        mlflow.log_param("solver", solver)
        mlflow.log_param("C", C)
        mlflow.set_tag("model type", "sklearn - LogisticRegression")
        lr.fit(x_train, y_train)
        precision, recall, accuracy = validate_model(lr, x_test, y_test)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.sklearn.log_model(lr, "model")
        print("Model saved in run %s" % mlflow.active_run().info.run_uuid)

And, for example, we can train it three times with different parameters to compare results.

In [ ]:
breast_cancer_lr()

In [ ]:
breast_cancer_lr(solver="liblinear")

In [ ]:
breast_cancer_lr(solver="liblinear", C=0.5)

The second model relies on Random Forest, which is another example of widely used machine learning model for both regression and classification.

In [ ]:
# Model 2: Random Forest
def breast_cancer_rf(n_estimators=100, max_depth=2, criterion="gini"):
    from sklearn.ensemble import RandomForestClassifier
    import mlflow.sklearn
    with mlflow.start_run() as run:
        clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, criterion=criterion)
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("criterion", criterion)
        mlflow.set_tag("model type", "sklearn - RandomForest")
        clf.fit(x_train, y_train)
        precision, recall, accuracy = validate_model(clf, x_test, y_test)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("accuracy", accuracy)
        mlflow.sklearn.log_model(clf, "model")
        print("Model saved in run %s" % mlflow.active_run().info.run_uuid)

Again, it is a good idea to train it three times with different parameters.

In [ ]:
breast_cancer_rf()

In [ ]:
breast_cancer_rf(max_depth=5)

In [ ]:
breast_cancer_rf(n_estimators=500, criterion="entropy")

Now, we're going to change the kind of model to show an interesting feature of MLFlow Tracking: tracking step.

A tracking step is a log that contains information about a specific iteration during the training process. For instance, in forward backward propagation algorithm to train neural network, several iterations (or epochs) of the same algorithm are execute in order to optimise the neural network weights step by step. The goal is to optimise the loss function to obtain the best effectiveness in the corresponding problem.

In this case, a MultiLayer Perceptron with a unique neuron as output is used. The class ``LossHistory`` is a callback class that defines the behaviour during the epochs of the training process. In this case, we are going to track loss, validation accuracy, and the so-called measures over the test set: precision, recall and accuracy.

In [ ]:
# Model 3: Keras
from keras.callbacks import Callback
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_epoch_end(self, epoch, logs={}):
        loss = logs.get('loss')
        acc = logs.get('accuracy')
        mlflow.log_metric("loss", loss, step=epoch)
        mlflow.log_metric("val_accuracy", acc, step=epoch)
        precision, recall, accuracy = validate_model(self.model, x_test, y_test)        
        mlflow.log_metric("precision", precision, step=epoch)
        mlflow.log_metric("recall", recall, step=epoch)
        mlflow.log_metric("accuracy", accuracy, step=epoch)
        self.losses.append(loss)
    

def breast_cancer_keras(optimizer='adam',dropout=0.00, nb_epoch=20):
    import mlflow.keras
    import keras
    from keras.models import Sequential
    from keras.layers import Dense, Dropout
    with mlflow.start_run() as run:
        mlflow.set_tag("model type", "keras - MLP")
        model = Sequential()
        # Adding the input layer and the first hidden layer
        model.add(Dense(output_dim=16, init='uniform', activation='relu', input_dim=30))
        # Adding dropout to prevent overfitting
        model.add(Dropout(p=dropout))
        # Adding the second hidden layer
        model.add(Dense(output_dim=16, init='uniform', activation='relu'))
        # Adding dropout to prevent overfitting
        model.add(Dropout(p=dropout))
        # Adding the output layer
        model.add(Dense(output_dim=1, init='uniform', activation='sigmoid'))
        # Compiling the ANN
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        history = LossHistory()
        model.fit(x_train, y_train, batch_size=100, nb_epoch=nb_epoch, callbacks=[history])
        mlflow.log_param("optimizer", optimizer)
        mlflow.log_param("dropout", dropout)
        mlflow.keras.log_model(model, "model")

As before, the model is trained three times with different parameters

In [ ]:
breast_cancer_keras()

In [ ]:
breast_cancer_keras(nb_epoch=100)

In [ ]:
breast_cancer_keras(optimizer='sgd', dropout=0.4, nb_epoch=100)

Now, the "model" is a little bit different: it is a custom code to compute the model output. In this case, we defined a random strategy to predict the output value, just to compare with the previous models.

This model must be defined as a method ``predict`` inside a class that inherits from ``mlflow.pyfunc.PythonModel``. PyFunc is the MLFlow API for custom models

In [ ]:
# Model 4: Custom model
import mlflow.pyfunc
from numpy import random
class CustomClassifier(mlflow.pyfunc.PythonModel):

    def predict(self, model_input):
        return np.random.randint(2, size=len(model_input))

Furthermore, we want now to save the dataset that was used to train the models, so we create with this method temporary files in numpy format.

In [ ]:
from tempfile import NamedTemporaryFile
def save_numpy_array(np_array):
    outfile = NamedTemporaryFile()
    np.save(outfile, np_array)
    return outfile

This model doesn't have a train execution so, we can directly log its results, the model, and the dataset's files.

In [ ]:
with mlflow.start_run() as run:
    ccl = CustomClassifier()
    precision, recall, accuracy = validate_model(ccl, x_test, y_test)
    mlflow.set_tag("model type", "pyfunc - random")
    mlflow.set_tag("dataset_uri", "https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)")
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("f1", (2 * precision * recall / (precision + recall))) # new metric for this model
    # Log custom model by means of pyfunc api
    mlflow.pyfunc.log_model("model", python_model=ccl)
    # Log dataset and splits used to train/test
    x_train_file = save_numpy_array(x_train)
    x_test_file = save_numpy_array(x_test)
    y_train_file = save_numpy_array(y_train)
    y_test_file = save_numpy_array(y_test)
    mlflow.log_artifact(x_train_file.name, "dataset/x_train")
    mlflow.log_artifact(x_test_file.name, "dataset/x_test")
    mlflow.log_artifact(y_train_file.name, "dataset/y_train")
    mlflow.log_artifact(y_test_file.name, "dataset/y_test")